In [1]:
jupyter:set_prolog_impl(swi).

true

In [23]:
dir_pos(n, Row-Col, Row1-Col):-
    Row1 is Row - 1.

dir_pos(e, Row-Col, Row-Col1):-
    Col1 is Col + 1.

dir_pos(s, Row-Col, Row1-Col):-
    Row1 is Row + 1.

dir_pos(w, Row-Col, Row-Col1):-
    Col1 is Col - 1.

is_pos_valid(Row-Col, MaxRow-MaxCol, Trees):-
    Row >= 1,
    Row =< MaxRow,
    Col >= 1,
    Col =< MaxCol,
    \+member(Row-Col, Trees).

check_tent_pos(MaxRow-MaxCol, Row-Col, Trees, [Dir | DirTail], [Dir | IL]):-
    check_tent_pos(MaxRow-MaxCol, Row-Col, Trees, DirTail, IL),
    dir_pos(Dir, Row-Col, TentRow-TentCol),
    is_pos_valid(TentRow-TentCol, MaxRow-MaxCol, Trees).

check_tent_pos(MaxRow-MaxCol, Row-Col, Trees, [Dir | DirTail], IL):-
    check_tent_pos(MaxRow-MaxCol, Row-Col, Trees, DirTail, IL),
    dir_pos(Dir, Row-Col, TentRow-TentCol),
    \+is_pos_valid(TentRow-TentCol, MaxRow-MaxCol, Trees).

    
check_tent_pos(_, _, _, [], []).

ensure_dir_lists([]).
ensure_dir_lists([H|T]):-
    length(H, L),
    L > 0,
    ensure_dir_lists(T).

iranylistak(MaxRow-MaxCol, Trees, IranyListak):-
    iranylistak(MaxRow-MaxCol, Trees, Trees, IranyListak),
    ensure_dir_lists(IranyListak), !.

iranylistak(_,_,[]).

iranylistak(MaxRow-MaxCol, [TreeRow-TreeCol | Fs], Trees, [IL| ILs]):-
    check_tent_pos(MaxRow-MaxCol, TreeRow-TreeCol, Trees, [e, n, s, w], IL),
    iranylistak(MaxRow-MaxCol, Fs, ILs).
iranylistak(_, [], _, []).

?- iranylistak(5-5, [1-1], ILs).
?- iranylistak(3-3, [1-1,2-1], ILs).
?- iranylistak(6-6, [1-3,1-5,3-1,4-6,5-1,5-4,6-5], ILs).
?- iranylistak(2-3, [1-1,1-2,2-1], ILs).

Previously defined clauses of user:dir_pos/3 were retracted:
:- dynamic dir_pos/3.

dir_pos(n, A-B, C-B) :-
    C is A-1.
dir_pos(e, A-B, A-C) :-
    C is B+1.
dir_pos(s, A-B, C-B) :-
    C is A+1.
dir_pos(w, A-B, A-C) :-
    C is B-1.

% Asserting clauses for user:dir_pos/3


Previously defined clauses of user:is_pos_valid/3 were retracted:
:- dynamic is_pos_valid/3.

is_pos_valid(A-B, C-D, E) :-
    A>=1,
    A=<C,
    B>=1,
    B=<D,
    \+ member(A-B, E).

% Asserting clauses for user:is_pos_valid/3


Previously defined clauses of user:check_tent_pos/5 were retracted:
:- dynamic check_tent_pos/5.

check_tent_pos(A-B, C-D, E, [F|G], [F|H]) :-
    is_pos_valid(I-J, A-B, E),
    dir_pos(F, C-D, I-J),
    check_tent_pos(A-B, C-D, E, G, H).
check_tent_pos(A-B, C-D, E, [F|G], H) :-
    dir_pos(F, C-D, I-J),
    \+ is_pos_valid(I-J, A-B, E),
    check_tent_pos(A-B, C-D, E, G, H).
check_tent_pos(_, _, _, [], []).

...

% Asserting clauses for user:iranylistak/4


ILs = [[e,s]]

ILs = [[e],[e,n,s]]

ILs = [[e,s,w],[e,s,w],[e,n,s],[n,s,w],[e,n,s],[e,n,s,w],[e,n,w]]

ILs = []